<a href="https://colab.research.google.com/github/theholymang8/Better-World-Working-Challenge/blob/main/Final_Notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#  !pip install folium
#  !pip install rasterio
#  !pip install pystac-client
#  !pip install planetary-computer
#  !pip install fsspec
#  !pip install requests
#  !pip install aiohttp
#  !pip install zarr
#  !pip install xarray[complete]

In [ ]:
# Supress Warnings 
import warnings
warnings.filterwarnings('ignore')

# Import common GIS tools
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import rasterio.features
import folium
import math

# Import Planetary Computer tools
import pystac_client
import planetary_computer

In [ ]:
# Greater Sydney, NSW
region_name = 'Greater Sydney, NSW'
min_lon, min_lat = (150.15, -34.25)  # Lower-left corner
max_lon, max_lat = (151.15, -33.25)  # Upper-right corner
bbox = (min_lon, min_lat, max_lon, max_lat)
latitude = (min_lat, max_lat)
longitude = (min_lon, max_lon)

In [ ]:
import pystac
collection = pystac.read_file("https://planetarycomputer.microsoft.com/api/stac/v1/collections/terraclimate")
asset = collection.assets["zarr-https"]

In [ ]:
import fsspec
import xarray as xr

store = fsspec.get_mapper(asset.href)
data = xr.open_zarr(store, **asset.extra_fields["xarray:open_kwargs"])

In [ ]:
clipped_data = data.sel(lon=slice(min_lon,max_lon),lat=slice(max_lat,min_lat),time=slice('2015-01-01','2019-12-31'))

In [ ]:
parsed_data = clipped_data[['tmax', 'tmin', 'ppt', 'soil']]


In [ ]:
import pandas as pd

In [ ]:
data_path = 'drive/MyDrive/Biodiversity Dataset/data-created-with-terraclimate.csv'

In [ ]:
gen_df = pd.read_csv(data_path)
gen_df

,max_of_max_temp,mean_of_min_temp,min_of_min_temp,mean_accumulated_precipitation,mean_soil_moisture,index
0,30.299999,12.2,4.3,85.199997,61.200001,935
1,31.299999,12.9,4.5,67.000000,53.299999,941
2,31.700001,10.6,2.5,68.000000,38.000000,944
3,31.700001,10.6,2.5,68.000000,38.000000,945
4,31.900000,12.2,3.8,63.099998,46.400002,980
...,...,...,...,...,...,...
9413,27.400000,7.1,0.0,73.400000,65.100000,193471
9414,30.400000,13.1,5.2,90.400000,72.600000,193473
9415,31.700000,10.6,2.5,68.000000,38.000000,193476
9416,28.200000,8.3,1.2,91.100000,62.700000,193477


In [ ]:
data_path = 'drive/MyDrive/Biodiversity Dataset/occurrence.txt'

In [ ]:
def filter_bbox(frogs, bbox):
    frogs = frogs[lambda x: 
        (x.decimalLongitude >= bbox[0]) &
        (x.decimalLatitude >= bbox[1]) &
        (x.decimalLongitude <= bbox[2]) &
        (x.decimalLatitude <= bbox[3])
    ]
    return frogs

def get_frogs(file, year_range=None, bbox=None):
    """Returns the dataframe of all frog occurrences for the bounding box specified."""
    columns = [
        'gbifID','eventDate','country','continent','stateProvince',
        'decimalLatitude','decimalLongitude','species'
    ]
    country_names = {
        'AU':'Australia', 'CR':'Costa Rica', 'ZA':'South Africa','MX':'Mexico','HN':'Honduras',
        'MZ':'Mozambique','BW':'Botswana','MW':'Malawi','CO':'Colombia','PA':'Panama','NI':'Nicaragua',
        'BZ':'Belize','ZW':'Zimbabwe','SZ':'Eswatini','ZM':'Zambia','GT':'Guatemala','LS':'Lesotho',
        'SV':'El Salvador', 'AO':'Angola', np.nan:'unknown or invalid'
    }
    continent_names = {
        'AU':'Australia', 'CR':'Central America', 'ZA':'Africa','MX':'Central America','HN':'Central America',
        'MZ':'Africa','BW':'Africa','MW':'Africa','CO':'Central America','PA':'Central America',
        'NI':'Central America','BZ':'Central America','ZW':'Africa','SZ':'Africa','ZM':'Africa',
        'GT':'Central America','LS':'Africa','SV':'Central America','AO':'Africa', np.nan:'unknown or invalid' 
    }
    frogs = (
        pd.read_csv(data_path, sep='\t', parse_dates=['eventDate'])
        .assign(
            country =  lambda x: x.countryCode.map(country_names),
            continent =  lambda x: x.countryCode.map(continent_names),
            species = lambda x: x.species.str.title()
        )
        [columns]
    )
    if year_range is not None:
        frogs = frogs[lambda x: 
            (x.eventDate.dt.year >= year_range[0]) & 
            (x.eventDate.dt.year <= year_range[1])
        ]
    if bbox is not None:
        frogs = filter_bbox(frogs, bbox)
    return frogs

In [ ]:
all_frog_data = get_frogs(data_path, year_range=(2015, 2019), bbox=bbox)
all_frog_data

,gbifID,eventDate,country,continent,stateProvince,decimalLatitude,decimalLongitude,species
935,3108894201,2017-11-12,Australia,Australia,New South Wales,-33.699881,151.043367,Litoria Fallax
941,3108882429,2019-09-19,Australia,Australia,New South Wales,-33.955790,150.976815,Crinia Signifera
944,3108953063,2019-11-03,Australia,Australia,New South Wales,-33.755278,150.623221,Litoria Fallax
945,3108952573,2019-11-04,Australia,Australia,New South Wales,-33.755291,150.623651,Litoria Fallax
980,1452200212,2015-06-16,Australia,Australia,New South Wales,-33.951844,150.870430,Litoria Fallax
...,...,...,...,...,...,...,...,...
193471,3108863070,2018-11-13,Australia,Australia,New South Wales,-33.474900,150.172000,Crinia Signifera
193473,3108900314,2019-10-30,Australia,Australia,New South Wales,-33.789908,151.131394,Litoria Fallax
193476,3108938525,2018-10-17,Australia,Australia,New South Wales,-33.757622,150.618240,Litoria Fallax
193477,3108845537,2019-10-15,Australia,Australia,New South Wales,-33.718800,150.385000,Crinia Signifera


In [ ]:
#gen_df['corelation_number'] = gen_df['index']
#gen_df.drop(axis = 1, columns='index', inplace=True)

gen_df.set_index('index', inplace=True)
gen_df

,max_of_max_temp,mean_of_min_temp,min_of_min_temp,mean_accumulated_precipitation,mean_soil_moisture
index,,,,,
935,30.299999,12.2,4.3,85.199997,61.200001
941,31.299999,12.9,4.5,67.000000,53.299999
944,31.700001,10.6,2.5,68.000000,38.000000
945,31.700001,10.6,2.5,68.000000,38.000000
980,31.900000,12.2,3.8,63.099998,46.400002
...,...,...,...,...,...
193471,27.400000,7.1,0.0,73.400000,65.100000
193473,30.400000,13.1,5.2,90.400000,72.600000
193476,31.700000,10.6,2.5,68.000000,38.000000


In [ ]:
all_frog_data['index'] = all_frog_data.index
all_frog_data.set_index('index', inplace=True)
all_frog_data

,gbifID,eventDate,country,continent,stateProvince,decimalLatitude,decimalLongitude,species
index,,,,,,,,
935,3108894201,2017-11-12,Australia,Australia,New South Wales,-33.699881,151.043367,Litoria Fallax
941,3108882429,2019-09-19,Australia,Australia,New South Wales,-33.955790,150.976815,Crinia Signifera
944,3108953063,2019-11-03,Australia,Australia,New South Wales,-33.755278,150.623221,Litoria Fallax
945,3108952573,2019-11-04,Australia,Australia,New South Wales,-33.755291,150.623651,Litoria Fallax
980,1452200212,2015-06-16,Australia,Australia,New South Wales,-33.951844,150.870430,Litoria Fallax
...,...,...,...,...,...,...,...,...
193471,3108863070,2018-11-13,Australia,Australia,New South Wales,-33.474900,150.172000,Crinia Signifera
193473,3108900314,2019-10-30,Australia,Australia,New South Wales,-33.789908,151.131394,Litoria Fallax
193476,3108938525,2018-10-17,Australia,Australia,New South Wales,-33.757622,150.618240,Litoria Fallax


In [ ]:
#all_gathered_data = pd.merge(all_frog_data, gen_df, on='index')

all_gathered_data

all_gathered_data.to_csv('drive/MyDrive/Biodiversity Dataset/frog_terraclimate_dataset.csv')

In [ ]:
#Custom Encoder
def customEncoder(value):
  if value == 'Litoria Fallax':
    return 1
  else:
    return 0

In [ ]:
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
enc = LabelEncoder()

Y = all_gathered_data.species
X = all_gathered_data.drop(columns={'eventDate', 'country', 'continent', 'stateProvince', 'gbifID', 'species'})

#Encode the species series
Y = [customEncoder(value) for value in Y]


In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.33, random_state=65)

#SMOTE implementation

In [ ]:
# Import SMOTE and make object
from imblearn.over_sampling import SMOTE
sampler = SMOTE(random_state=0)

In [ ]:
# SMOTE New X_train and New y_train variables
X_train_smote, y_train_smote = sampler.fit_resample(X_train,y_train)

#OSS Implementation Minority Class Undersample


In [ ]:
from imblearn.under_sampling import OneSidedSelection

undersample = OneSidedSelection(n_neighbors=1, n_seeds_S=100)

X_train_oss, y_train_oss = undersample.fit_resample(X_train, y_train)

#SMOTE Implementation Majority Class Decrease

In [ ]:
from imblearn.combine import SMOTEENN
smt = SMOTEENN(random_state=42)
X_train_smoteen, y_train_smoteen = smt.fit_resample(X_train,y_train)

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score, accuracy_score, ConfusionMatrixDisplay
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import KFold

In [ ]:
# Machine Learning
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score, accuracy_score, ConfusionMatrixDisplay
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import KFold

#KNN Neighbors

In [ ]:
def knnClassifier(X_train, y_train, X_test, y_test):
  knn_model = KNeighborsClassifier(
      algorithm= 'ball_tree',
      leaf_size= 10,
      metric='euclidean',
      n_neighbors= 5,
      weights= 'distance'
  )

  knn_model.fit(X_train,y_train) # Smote is applied only to training datasets

  y_predict_knn = knn_model.predict(X_test)

  print('Accuracy Score: {}'.format(accuracy_score(y_test,y_predict_knn)))
  print('F1 Score: {}'.format(f1_score(y_predict_knn, y_test, average='binary')))
  #cv=KFold(n_splits=10, random_state=1, shuffle=True)
  #use k-fold CV to evaluate model
  scores = cross_val_score(knn_model, X_test, y_test, scoring='neg_mean_absolute_error',
                          cv=KFold(n_splits=10, random_state=1, shuffle=True), n_jobs=-1)


  #view mean absolute error
  print('CV Score: {}'.format(mean(absolute(scores))))




In [ ]:
#kf = KFold(4, n_folds=2)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

knnClassifier(X_train_smoteen, y_train_smoteen, X_test, y_test)

Accuracy Score: 0.8008365508365508
F1 Score: 0.7119590507212656
CV Score: 0.18050513432216575


In [ ]:
from sklearn.model_selection import GridSearchCV

grid_search = GridSearchCV(KNeighborsClassifier(),
                           {
                              'n_neighbors':np.arange(5, 100, 5),
                              'algorithm': ['ball_tree', 'kd_tree', 'brute'],
                              'weights': ['uniform', 'distance'],
                              'leaf_size': np.arange(10, 50, 10),
                              #'p': [1, 2],
                              'metric': ['euclidean', 'seuclidean']
                              #'max_features':np.arange(0.1,1.0,0.05),
                            
                            },cv=5, scoring="r2",verbose=1,n_jobs=-1
                           )
grid_search.fit(X_train_smoteen,y_train_smoteen)

grid_search.best_params_

Fitting 5 folds for each of 912 candidates, totalling 4560 fits


{'algorithm': 'ball_tree',
 'leaf_size': 10,
 'metric': 'euclidean',
 'n_neighbors': 5,
 'weights': 'distance'}

KFold Check for Model Overfitting

In [ ]:
from sklearn.model_selection import cross_val_score
from numpy import mean
from numpy import absolute
from numpy import sqrt

cv = KFold(n_splits=10, random_state=1, shuffle=True)

#use k-fold CV to evaluate model
scores = cross_val_score(knn_model, X_test, y_test, scoring='neg_mean_absolute_error',
                         cv=cv, n_jobs=-1)


#view mean absolute error
mean(absolute(scores))

0.2010963592988279

#Logistic Regression

In [ ]:
from sklearn.linear_model import LogisticRegression

logistic_regression_model = LogisticRegression(
    solver = 'liblinear',
    fit_intercept = True,
    C = 1.0
)

logistic_regression_model.fit(X_train_oss, y_train_oss)

Y_predict_logistic = logistic_regression_model.predict(
    X_test
)

print(accuracy_score(y_test,Y_predict_logistic))
print(f1_score(Y_predict_logistic, y_test, average='binary'))

0.7474259974259975
0.5352279455298994


In [ ]:
scores = cross_val_score(logistic_regression_model, X_test, y_test, scoring='neg_mean_absolute_error',
                         cv=cv, n_jobs=-1)


#view mean absolute error
mean(absolute(scores))

0.23775956850949073

#Random Forest Classifier

In [ ]:
from sklearn.ensemble import RandomForestClassifier

RFC = RandomForestClassifier(
    n_estimators=100,
    min_samples_leaf=2
    #ccp_alpha=0.0 #By default no pruning occurs.
)

RFC.fit(X_train_oss, y_train_oss)

Y_predict_RFC = RFC.predict(
    X_test
)

print(accuracy_score(y_test,Y_predict_RFC))
print(f1_score(Y_predict_RFC, y_test, average='binary'))

0.8194980694980695
0.7136294027565083


In [ ]:
scores = cross_val_score(RFC, X_test, y_test, scoring='neg_mean_absolute_error',
                         cv=cv, n_jobs=-1)


#view mean absolute error
mean(absolute(scores))

0.18404522352453065